In [1]:
import numpy as np
import pylab as pl
import arrayfire as af
af.set_backend("cpu")

In [2]:
pl.rcParams['figure.figsize']  = 12, 7.5
pl.rcParams['lines.linewidth'] = 1.5
pl.rcParams['font.family']     = 'serif'
pl.rcParams['font.weight']     = 'bold'
pl.rcParams['font.size']       = 20  
pl.rcParams['font.sans-serif'] = 'serif'
pl.rcParams['text.usetex']     = True
pl.rcParams['axes.linewidth']  = 1.5
pl.rcParams['axes.titlesize']  = 'medium'
pl.rcParams['axes.labelsize']  = 'medium'

pl.rcParams['xtick.major.size'] = 8     
pl.rcParams['xtick.minor.size'] = 4     
pl.rcParams['xtick.major.pad']  = 8     
pl.rcParams['xtick.minor.pad']  = 8     
pl.rcParams['xtick.color']      = 'k'     
pl.rcParams['xtick.labelsize']  = 'medium'
pl.rcParams['xtick.direction']  = 'in'    

pl.rcParams['ytick.major.size'] = 8     
pl.rcParams['ytick.minor.size'] = 4     
pl.rcParams['ytick.major.pad']  = 8     
pl.rcParams['ytick.minor.pad']  = 8     
pl.rcParams['ytick.color']      = 'k'     
pl.rcParams['ytick.labelsize']  = 'medium'
pl.rcParams['ytick.direction']  = 'in' 

In [3]:
# Setting velocity and spatial grid points
N_positions = 500
ghost_zones = 3
N_velocity  = 500

In [4]:
# Boundaries of domain
left_boundary  = 0
right_boundary = 1.0
length         = right_boundary - left_boundary

In [5]:
# Setting mass of the particle, boltzmann-constant
mass_particle      = 1.0
boltzmann_constant = 1.0

In [6]:
# Scattering time scale
tau   = 0.05
# Magnitude of maximum velocity
v_max = 5.0

In [7]:
# Time Parameters for the simulation:
dt         = 0.005 # Size of the time-step
final_time = 10.0
time       = np.arange(dt, final_time, dt)

In [8]:
# Setting up the temperature parameters for the simulations:
T_left   = 1.0
T_right  = 1.0

In [9]:
# Setting up of the density parameters for the simulation:
rho_left  = 1.0
rho_right = 1.0

In [10]:
# Bulk velocity parameters for left and right states:
v_bulk_l = 0
v_bulk_r = 0

In [11]:
# Setting up of spatial and velocity grids:
x  = np.linspace(left_boundary, right_boundary, N_positions)
dx = x[1] - x[0]

In [12]:
# Obtaining the coordinates for the ghost-zones:
x_ghost_left  = np.linspace(-(ghost_zones)*dx + left_boundary, left_boundary - dx, ghost_zones)
x_ghost_right = np.linspace(right_boundary + dx, right_boundary + ghost_zones*dx , ghost_zones)

In [16]:
# Combining them to obtain the entire spatial grid
x  = np.concatenate([x_ghost_left, x, x_ghost_right])

In [17]:
# Obtaining the velocity grid
v  = np.linspace(-v_max, v_max, N_velocity)
x  = af.to_array(x)
v  = af.to_array(v)

v_bulk = af.constant(0, N_positions + 2*ghost_zones, N_velocity)
v_bulk[:ghost_zones + 0.5*N_positions, :] = v_bulk_l
v_bulk[ghost_zones + 0.5*N_positions:, :] = v_bulk_r

In [18]:
# Conversion to allow for easy vectorization
x = af.tile(x, 1, N_velocity)
v = af.tile(af.reorder(v), N_positions + 2*ghost_zones, 1)

In [19]:
def calculate_density(f, v):
    deltav           = af.sum(v[0, 1]-v[0, 0])
    value_of_density = af.sum(f, 1)*deltav
    af.eval(value_of_density)
    return(value_of_density)

In [17]:
def calculate_momentum(f, v):
    deltav            = af.sum(v[0, 1]-v[0, 0])
    value_of_momentum = af.sum(f*v, 1)*deltav
    af.eval(value_of_momentum)
    return(value_of_momentum)

In [18]:
def calculate_temperature(f, v):
    deltav               = af.sum(v[0, 1]-v[0, 0])
    value_of_temperature = af.sum(f*(v - v_bulk)**2, 1)*deltav
    value_of_temperature = value_of_temperature/calculate_density(f, v)
    af.eval(value_of_temperature)
    return(value_of_temperature)

In [19]:
def f_MB(x, v, f):
    n = af.tile(calculate_density(f, v), 1, N_velocity)
    T = af.tile(calculate_temperature(f, v), 1, N_velocity)
    f_MB = n*af.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T))*\
             af.exp(-mass_particle*(v - v_bulk)**2/(2*boltzmann_constant*T))
    af.eval(f_MB)
    return(f_MB)


In [20]:
def f_interp(dt, x, v, f):
    x_new     = x - v*dt
    step_size = af.sum(x[1,0] - x[0,0])
    f_inter   = af.constant(0, N_positions + 2*ghost_zones, N_velocity)
    
    f_inter   = af.Array.as_type(f_inter, af.Dtype.f64)
    
    x_new[ghost_zones:-ghost_zones, :] = af.select(x_new[ghost_zones:-ghost_zones, :]<=left_boundary, x_new[ghost_zones:-ghost_zones, :] + length, x_new[ghost_zones:-ghost_zones, :])
    x_new[ghost_zones:-ghost_zones, :] = af.select(x_new[ghost_zones:-ghost_zones, :]>=right_boundary, x_new[ghost_zones:-ghost_zones, :] - length, x_new[ghost_zones:-ghost_zones, :])
    
    f_inter[ghost_zones:-ghost_zones,:] = af.approx1(f, (x_new[ghost_zones:-ghost_zones,:]/step_size), af.INTERP.CUBIC)
    
    af.eval(f_inter)
    return f_inter


In [21]:
# Intializing the values for f
f_initial = af.constant(0, N_positions + 2*ghost_zones, N_velocity)
rho       = 0.1*af.exp(-10*(x-0.5)**2) + 1

In [22]:
f_initial = rho * np.sqrt(mass_particle/(2*np.pi*boltzmann_constant*T_left))*\
                  af.exp(-mass_particle*(v - v_bulk_l)**2/(2*boltzmann_constant*T_left))

In [23]:
f_initial[:ghost_zones, :] = \
f_initial[N_positions - 2:N_positions - 2 + ghost_zones, :]

f_initial[ghost_zones + N_positions:, :] = \
f_initial[1 + ghost_zones:1 + 2*ghost_zones, :]

In [25]:
# Initializing the starting value for the simulation
f_current = f_initial
print((calculate_momentum(f_initial, v)))
pl.xlabel('$x$')
pl.ylabel(r'$\rho$')
pl.plot(x[ghost_zones:-ghost_zones, 0], calculate_density(f_current, v)[ghost_zones:-ghost_zones, :])
pl.title('Initial Conditions')
pl.ylim([0.8, 1.2])
pl.savefig('images/0000.png')
pl.clf()

arrayfire.Array()
Type: double
[506 1 1 1]
   -0.0000 
    0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
    0.0000 
    0.0000 
    0.0000 
    0.0000 
    0.0000 
   -0.0000 
    0.0000 
    0.0000 
   -0.0000 
    0.0000 
    0.0000 
    0.0000 
   -0.0000 
    0.0000 
    0.0000 
    0.0000 
   -0.0000 
    0.0000 
    0.0000 
    0.0000 
    0.0000 
    0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
    0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
    0.0000 
    0.0000 
    0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
    0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
    0.0000 
    0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
    0.0000 
    0.0000 
    0.0000 
   -0.0000 
   -0.000

In [29]:
for time_index, t0 in enumerate(time):
    print("Computing For Time Index = ", time_index)
    #print("Physical Time            = ", t0)
    # We shall split the Boltzmann-Equation and solve it:
    # In this step we are solving the collisionless equation
    f_current = af.Array.as_type(f_current, af.Dtype.f64)
    fstar = f_interp(dt, x, v, f_current)
    
    fstar[:ghost_zones, :] = \
    fstar[N_positions - 2:N_positions - 2 + ghost_zones, :]

    fstar[ghost_zones + N_positions:, :] = \
    fstar[1 + ghost_zones:1 + 2*ghost_zones, :]
    
    # We turn off the term v(df/dx) for the following two steps
    f0             = f_MB(x, v, fstar)
    f_new          = f0 + (fstar - f0)*np.exp(-dt/tau)
    
    f_new[:ghost_zones, :] = \
    f_new[N_positions - 2:N_positions - 2 + ghost_zones, :]

    f_new[ghost_zones + N_positions:, :] = \
    f_new[1 + ghost_zones:1 + 2*ghost_zones, :]

    f_current = f_new
    mom = calculate_momentum(f_initial, v)
    print(af.sum(((af.sign(mom)*-1+1)-af.sign(mom))*mom))
    pl.xlabel('$x$')
    pl.ylabel(r'$\rho$')
    pl.plot(x[ghost_zones:-ghost_zones, :], calculate_density(f_current, v)[ghost_zones:-ghost_zones, :])
    pl.title('Time = ' + str(t0))
    pl.ylim([0.8, 1.2])
    pl.savefig('images/' + '%04d'%(time_index + 1) + '.png')
    pl.clf()

Computing For Time Index =  0
4.07248632812365e-14
Computing For Time Index =  1
4.07248632812365e-14
Computing For Time Index =  2
4.07248632812365e-14
Computing For Time Index =  3
4.07248632812365e-14


KeyboardInterrupt: 